# 文本分類

In [1]:
!pip install transformers

## 01 載入相關套件

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## 02 載入數據

# ChnSentiCorp_htl_all	7000 中國酒店評論數據，5000 正向評論，2000 負向評論

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
torch.cuda.is_available()

True

In [5]:
!nvidia-smi

Sat Dec  9 12:46:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import pandas as pd


data = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/NLP_tutorial/Transformers 大祕寶/transformers-code/01-Getting Started/04-model/ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7766 entries, 0 to 7765
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   7766 non-null   int64 
 1   review  7765 non-null   object
dtypes: int64(1), object(1)
memory usage: 121.5+ KB


## 03 建立dataset (資料讀取器) - 備料


### PyTorch中的dataset範例

在PyTorch中，`Dataset`是一个抽象類別，用來表示數據集。您可以根據自己的數據格式創建自定義的`Dataset`類，有效加載和處理数据。以下是一個簡單範例，展示如何创建一个自定義的PyTorch `Dataset`类。



import torch
from torch.utils.data import Dataset

```python
class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        sample = self.data[index]
        target = self.targets[index]
        
        return sample, target


In [8]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__() #父類初始化
        self.data = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/NLP_tutorial/Transformers 大祕寶/transformers-code/01-Getting Started/04-model/ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]

    def __len__(self):
        return len(self.data)


In [9]:
# dataset 本身為一個可以迭代的物件
dataset = MyDataset()
for i in range(10):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)
('总的来说，这样的酒店配这样的价格还算可以，希望他赶快装修，给我的客人留些好的印象', 1)
('价格比比较不错的酒店。这次免费升级了，感谢前台服务员。房子还好，地毯是新的，比上次的好些。早餐的人很多要早去些。', 1)
('不错，在同等档次酒店中应该是值得推荐的！', 1)
('入住丽晶，感觉很好。因为是新酒店，的确有淡淡的油漆味，房间内较新。房间大小合适，卫生间设备齐全，服务态度也很好。网速可能是提升了吧，感觉还好。有免费的插线板、水果刀。。。等一系列的日常用品免费提供，很细心。晚上的自助餐是每人68元，菜品一般、就算说的过去吧。酒店外有西贝筱面等几家饭馆。西贝的菜实在是量大的惊人，人多在酒店吃就不如去西贝吃了。酒店21：30-24点有广式晚茶，吃过几次，味道正宗，就是点心品种略少。哈。酒店地处青山区，包头分昆区、青山区。相对来说：昆区是最繁华的，青山区次之。像包百等大商场全在昆区，从酒店打车到包百商圈车费12元多。青山区也有一个王府井百货，相对来说比包百商圈的略小，从酒店打车8元多。银河广场离酒店有一段距离，打车9、10元吧，银河广场边上有“科丽珑”24小时营业的超市。酒店一楼有小卖部，周围基本没有超市。这次住正赶上有几家公司在这里开会，有时楼道里略吵，晚上还有几次有人敲错门，理论上与酒店管理没关系吧。希望以后还是尽量让会议团体住在一层，也许这样可以好些吧。总体感觉丽晶很好，以后来包头还会住这里。', 1)
('1。酒店比较新，装

## 04 切分數據集 (訓練, 驗證)

In [10]:
from torch.utils.data import random_split

trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [11]:
for i in range(10):
    print(trainset[i])

('我第一次入住如家，实际情况与现实相差太远，屋子里味道很大，打开窗户换气又很吵，卫生不敢恭维！入住时间不长就搬走。而且位置也很偏，不知道为何在这里选址！总之不会再住！楼下涮肉一般！', 0)
('住宿环境还是不错的，比较干净。前台的服务态度很好，一个小伙子，我们说想k歌，就带我们过去看房间，还告诉我怎么吃饭比较方便。说起吃饭，我建议就在酒店的餐厅吃，比外面所谓的美食街贵不了多少，我是有切身感受的。还有就是自驾的，我最想不通的就是自己开车去住店，居然收我20元停车费，这个简直太没有道理了，有点想钱想疯的感觉！！！', 1)
('房间设施还是很先进的，自动化程度相当高。但是我最喜欢的还是演艺馆的演出。竟然看了6场。水平真得不错，号称那些歌舞演员也是太子的正式员工，也要打卡上班的呢。杂技就更加精彩了，还有几个得过世界大奖的节目。建议去东莞出差的朋友一定不要放过这个机会，演艺馆才是最具特色的。对了，别忘了提前预订座位，中间的，前排的，最低消费也不高，每人才60多块钱，从晚上9点15看到12点，很值~~', 1)
('总台的服务生挺热情的，感觉很亲切。这家酒店最大的特点是房间宽敞，床很软，睡得舒服，免费宽带上网对我很实用。离机场的距离不远，到我办事的公司也近，下次还会选择入住。', 1)
('非常不行，三星的携程定290元，说景观房，其实什么都看不到，连大马路都不繁华。火车站附近佳丽4星，协议价298。只能说这变价高了，而且很老破破的样子，一点花园的感觉都没！其余一般。', 0)
('我是7月上旬去过的，酒店的环境很适合度假。山清水秀，幽静非常。但房间就不敢恭维；一进房首先是地毯的霉味很大很大（我要求换房可是很多标间都一样），令人顶不顺。再有是灯光很暗，和房门锁不牢。还好服务员的服务意识和态度不错。值得安慰', 1)
('个人认为不符合4星的条件,房间很潮湿,酶味太重,,根本没办法入睡,浴室的条件也不怎么样,总之感觉更像个快捷式酒店', 0)
('交通很便利，离商业街（太原街）很近，虽然太原街破了点，但还不错，离车站也很近。房间比较安静。面积不是太大。但床挺舒服。服务也还好，但赶紧价位有点高啊。总体感觉值得入住。', 1)
('酒店就在西门长途汽车站对面，因此如果坐长途汽车过去，到西门站下，马路对面就是。大门实际在环城西路，不过指示牌很明显，因此在人民中路也可以看到。豪华标

## 05 建立 dataloader (資料打包器) - 前處理

In [12]:
# 只下載 pytorch model
!git lfs clone "https://huggingface.co/hfl/rbt3" --include="*.bin" ./rbt3

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into './rbt3'...
remote: Enumerating objects: 48, done.
remote: Total 48 (delta 0), reused 0 (delta 0), pack-reused 48
Unpacking objects: 100% (48/48), 156.45 KiB | 4.89 MiB/s, done.


In [13]:
import torch

# 定義分詞器
tokenizer = AutoTokenizer.from_pretrained("rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

In [14]:
from torch.utils.data import DataLoader

"""
pipeline邏輯:
pytorch 透過 dataset 去random split 數據集(訓練和驗證)
dataloader 對數據集進行資料轉換(token化 + 批次 and 洗牌)
"""

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [15]:
# 查看第0個batch的輸出項目
next(enumerate(validloader))[1].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [16]:
# input_ids
print('The dimensions of the input_ids tensor are generally in the format [batch_size, sequence_length]')

print(next(enumerate(validloader))[1].get("input_ids"))

print(next(enumerate(validloader))[1].get("token_type_ids").shape)

The dimensions of the input_ids tensor are generally in the format [batch_size, sequence_length]
tensor([[ 101, 2769,  857,  ...,    0,    0,    0],
        [ 101,  679, 6421,  ...,    0,    0,    0],
        [ 101, 1057,  857,  ...,    0,    0,    0],
        ...,
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101, 1429, 1357,  ..., 8024, 3241,  102],
        [ 101, 3221, 1103,  ...,    0,    0,    0]])
torch.Size([64, 128])


In [17]:
# token_type_ids
print('The dimensions of the input_ids tensor are generally in the format [batch_size, sequence_length]')

print(next(enumerate(validloader))[1].get("token_type_ids"))

print(next(enumerate(validloader))[1].get("token_type_ids").shape)


The dimensions of the input_ids tensor are generally in the format [batch_size, sequence_length]
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
torch.Size([64, 128])


In [18]:
# attention_mask
print('The dimensions of the attention_mask tensor are generally in the format [batch_size, sequence_length]')

print(next(enumerate(validloader))[1].get("attention_mask"))

print(next(enumerate(validloader))[1].get("attention_mask").shape)

The dimensions of the attention_mask tensor are generally in the format [batch_size, sequence_length]
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])
torch.Size([64, 128])


In [19]:
# labels
print('The dimensions of the labels tensor are generally in the format [batch_size]')

print(next(enumerate(validloader))[1].get("labels"))

print(next(enumerate(validloader))[1].get("labels").shape)


The dimensions of the labels tensor are generally in the format [batch_size]
tensor([1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1])
torch.Size([64])


`Classification` Tasks: For tasks like sentiment analysis, where each input (such as a sentence or document) is classified into categories, the labels tensor often has the dimensions [batch_size]. Each element in this tensor is a single integer representing the class label for the corresponding input.

`Token-Level Tasks`: For tasks like named entity recognition or part-of-speech tagging, where each token in a sequence is labeled, the labels tensor might indeed have dimensions [batch_size, sequence_length]. Each element in this tensor is an integer representing the label for the corresponding token.

`Sequence-to-Sequence Tasks`: For tasks like machine translation, the labels tensor (which represents the target sequences) could have dimensions [batch_size, target_sequence_length], where target_sequence_length could be different from the input_sequence_length.

##06 模型創建及設定調參優化器

In [20]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("./rbt3/")

if torch.cuda.is_available():
    model = model.cuda()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./rbt3/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# 標準寫法
optimizer = Adam(model.parameters(), lr=2e-5)

##07 定義訓練和驗證方程

In [29]:
for batch in validloader:
    print(batch.keys())
    break


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


### 驗證

1. 切換評估模式
2. 推論
3. 計算正確率

In [31]:
def evaluate():
    model.eval() #
    acc_num = 0
    with torch.inference_mode(): #
        for batch in validloader:
            if torch.cuda.is_available():
                # print("GPU mode")
                # 將 tensor 導入cuda 進行 GPU 計算
                batch = {k: v.cuda() for k, v in batch.items()}

            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            # 通常 pred 和 label 的位元數不一樣，透過.long 統一轉換為 64 維的整數
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

### 訓練

1. 切換訓練模式
2. 權重初始化
3. 輸入訓練
4. 計算 loss
5. 反向傳播計算權重
6. 更新權重

In [32]:
def train(epoch=3, log_step=100):
    if torch.cuda.is_available():
        print("GPU is ok")
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                # print("GPU mode")
                batch = {k: v.cuda() for k, v in batch.items()}
            # 梯度初始化
            optimizer.zero_grad()
            # 訓練
            output = model(**batch)
            # 反向傳播紀錄損失函數的梯度
            output.loss.backward()
            # 透過優化器更新模型參數
            optimizer.step()
            if global_step % log_step == 0:
                print(f"epoch: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"epoch: {ep}, acc: {acc}")



## 09 模型訓練

In [33]:
train()

GPU is ok
epoch: 0, global_step: 0, loss: 0.17382517457008362
epoch: 0, global_step: 100, loss: 0.25520724058151245
epoch: 0, global_step: 200, loss: 0.17781750857830048
epoch: 0, acc: 0.9020618200302124
epoch: 1, global_step: 300, loss: 0.14967548847198486
epoch: 1, global_step: 400, loss: 0.4859044849872589
epoch: 1, acc: 0.8865979313850403
epoch: 2, global_step: 500, loss: 0.09773145616054535
epoch: 2, global_step: 600, loss: 0.063675656914711
epoch: 2, acc: 0.894329845905304


## 具備進度條功能

In [34]:
from tqdm import tqdm

def train(epoch=3, log_step=100):
    if torch.cuda.is_available():
        print("GPU is ok")
    global_step = 0
    for ep in range(epoch):
        model.train()
        # Wrap your data loader with tqdm for a progress bar
        progress_bar = tqdm(trainloader, desc=f"Epoch {ep+1}/{epoch}")
        for batch in progress_bar:
            if torch.cuda.is_available():
                # print("GPU mode")
                batch = {k: v.cuda() for k, v in batch.items()}

            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()

            if global_step % log_step == 0:
                # Update the progress bar instead of printing
                progress_bar.set_postfix(loss=output.loss.item())

            global_step += 1
        acc = evaluate()
        # Print accuracy outside the progress bar
        print(f"Epoch {ep+1}/{epoch}, Accuracy: {acc}")


In [35]:
train()

GPU is ok


Epoch 1/3:  15%|█▍        | 32/219 [00:06<00:37,  4.94it/s, loss=0.0624]


KeyboardInterrupt: ignored

## 10 模型預測

In [39]:
sen = "這家飯店的廁所有煙味！衣櫃有蟑螂"

id2_label = {0: "bad！", 1: "good！"}
model.eval()

with torch.inference_mode():

    inputs = tokenizer(sen, return_tensors="pt")

    inputs = {k: v.cuda() for k, v in inputs.items()}
    # model = AutoModelForSequenceClassification.from_pretrained("./rbt3/")
    logits = model(**inputs).logits

    pred = torch.argmax(logits, dim=-1)

    print(f"输入：{sen}\n模型預測結果 : {id2_label.get(pred.item())}")



输入：這家飯店的廁所有煙味！衣櫃有蟑螂
模型預測結果 : bad！


# 簡易寫法

In [40]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

sen_2 = "飯店生蠔，臭酸"
pipe(sen_2)



[{'label': 'bad！', 'score': 0.5611153244972229}]